In [0]:
!pip install optuna

In [0]:
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras import models
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.utils import np_utils
import numpy as np
import optuna

'''
データの設定
'''
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [109]:
test_images.shape

(10000, 28, 28, 1)

In [0]:
'''
モデルの構築
'''   
callbacks_list = [
    ModelCheckpoint(
        filepath="my_model.h5",     # モデルの保存先となるファイルのパス
        monitor='val_loss',         # val_lossを監視
        save_best_only=True,        # val_lossが改善した時のみ保存
    )
]
def create_model(n_layer, activation, mid_units, optimizer, dropout_rate):
    model = models.Sequential()
    for i in range(n_layer):
        model.add(
            Conv2D(mid_units, (3, 3), 
            activation=activation,
            input_shape= (28, 28, 1),
            padding = "same"
        ))
        model.add(MaxPooling2D((2, 2),padding="same"))
        model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(mid_units, activation= activation ))
    model.add(Dropout(dropout_rate))
    model.add(Dense(10, activation= "softmax"))
    return model
  

In [0]:
'''
学習
'''
def objective(trial):
    n_layer = trial.suggest_int('n_layer', 1, 6)
    activation = trial.suggest_categorical('activation', ['relu', 'sigmoid']) 
    mid_units = int(trial.suggest_discrete_uniform('mid_units', 2, 128, 2)) # ユニット数
    optimizer = trial.suggest_categorical('optimizer', ['sgd', 'adam', 'rmsprop'])
    dropout_rate = trial.suggest_uniform('dropout_rate', 0, 1)
    
    model = create_model(n_layer, activation, mid_units, optimizer, dropout_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
    history = model.fit(
        train_images, train_labels,
        epochs=10,
        batch_size= 4096,
        validation_data=(test_images, test_labels),
        callbacks = callbacks_list
    )
    
    #検証用データに対する正答率が最大となるハイパーパラメータを求める
    return 1 - history.history["val_acc"][-1]
    

In [197]:
def main():
    study = optuna.create_study(sampler=optuna.samplers.TPESampler())
    study = optuna.create_study(sampler=optuna.samplers.TPESampler())
    study.optimize(objective, n_trials=10)
    print('best_params')
    print(study.best_params)
    print('-1 x best_value')
    print(-study.best_value)

    print('\n --- sorted --- \n')
    sorted_best_params = sorted(study.best_params.items(), key=lambda x : x[0])
    for i, k in sorted_best_params:
        print(i + ' : ' + str(k))


if __name__ == '__main__':
    main()

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 24s 399us/step - loss: 0.8735 - acc: 0.7264 - val_loss: 0.2179 - val_acc: 0.9352
Epoch 2/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.1713 - acc: 0.9471 - val_loss: 0.1141 - val_acc: 0.9645
Epoch 3/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0973 - acc: 0.9702 - val_loss: 0.0707 - val_acc: 0.9788
Epoch 4/10
60000/60000 [==============================] - 4s 66us/step - loss: 0.0687 - acc: 0.9793 - val_loss: 0.0535 - val_acc: 0.9834
Epoch 5/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0538 - acc: 0.9839 - val_loss: 0.0490 - val_acc: 0.9857
Epoch 6/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0428 - acc: 0.9875 - val_loss: 0.0403 - val_acc: 0.9880
Epoch 7/10
60000/60000 [==============================] - 4s 67us/step - loss: 0.0367 - acc: 0.9896 - val_loss: 0.0421 - val_a

[I 2019-04-23 12:27:39,783] Finished trial#0 resulted in value: 0.011000003910064726. Current best value is 0.011000003910064726 with parameters: {'n_layer': 3, 'activation': 'relu', 'mid_units': 88.0, 'optimizer': 'adam', 'dropout_rate': 0.0994584167844218}.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 19s 324us/step - loss: 2.2171 - acc: 0.2239 - val_loss: 1.3464 - val_acc: 0.6774
Epoch 2/10
60000/60000 [==============================] - 2s 32us/step - loss: 1.6772 - acc: 0.3675 - val_loss: 0.7999 - val_acc: 0.8423
Epoch 3/10
60000/60000 [==============================] - 2s 32us/step - loss: 1.3761 - acc: 0.4773 - val_loss: 0.5107 - val_acc: 0.9179
Epoch 4/10
60000/60000 [==============================] - 2s 33us/step - loss: 1.1872 - acc: 0.5356 - val_loss: 0.3614 - val_acc: 0.9460
Epoch 5/10
60000/60000 [==============================] - 2s 33us/step - loss: 1.0712 - acc: 0.5739 - val_loss: 0.2703 - val_acc: 0.9560
Epoch 6/10
60000/60000 [==============================] - 2s 33us/step - loss: 0.9926 - acc: 0.6031 - val_loss: 0.2135 - val_acc: 0.9616
Epoch 7/10
60000/60000 [==============================] - 2s 33us/step - loss: 0.9231 - acc: 0.6310 - val_loss: 0.1778 - val_a

[I 2019-04-23 12:28:19,395] Finished trial#1 resulted in value: 0.026600001335144063. Current best value is 0.011000003910064726 with parameters: {'n_layer': 3, 'activation': 'relu', 'mid_units': 88.0, 'optimizer': 'adam', 'dropout_rate': 0.0994584167844218}.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 18s 303us/step - loss: 2.4583 - acc: 0.1270 - val_loss: 2.0522 - val_acc: 0.2275
Epoch 2/10
60000/60000 [==============================] - 1s 16us/step - loss: 2.2256 - acc: 0.1679 - val_loss: 1.9064 - val_acc: 0.3362
Epoch 3/10
60000/60000 [==============================] - 1s 15us/step - loss: 2.1137 - acc: 0.1942 - val_loss: 1.8166 - val_acc: 0.4726
Epoch 4/10
60000/60000 [==============================] - 1s 15us/step - loss: 2.0358 - acc: 0.2165 - val_loss: 1.7471 - val_acc: 0.5897
Epoch 5/10
60000/60000 [==============================] - 1s 15us/step - loss: 1.9684 - acc: 0.2400 - val_loss: 1.6924 - val_acc: 0.7203
Epoch 6/10
60000/60000 [==============================] - 1s 15us/step - loss: 1.9102 - acc: 0.2640 - val_loss: 1.6375 - val_acc: 0.7894
Epoch 7/10
60000/60000 [==============================] - 1s 15us/step - loss: 1.8629 - acc: 0.2825 - val_loss: 1.5893 - val_a

...

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 22s 370us/step - loss: 1.0429 - acc: 0.6747 - val_loss: 0.4290 - val_acc: 0.8875
Epoch 2/10
60000/60000 [==============================] - 3s 52us/step - loss: 0.4473 - acc: 0.8738 - val_loss: 0.3140 - val_acc: 0.9174
Epoch 3/10
60000/60000 [==============================] - 3s 52us/step - loss: 0.3649 - acc: 0.8958 - val_loss: 0.2723 - val_acc: 0.9247
Epoch 4/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.3184 - acc: 0.9096 - val_loss: 0.2414 - val_acc: 0.9335
Epoch 5/10
60000/60000 [==============================] - 3s 51us/step - loss: 0.2908 - acc: 0.9172 - val_loss: 0.2230 - val_acc: 0.9385
Epoch 6/10
60000/60000 [==============================] - 3s 52us/step - loss: 0.2661 - acc: 0.9240 - val_loss: 0.2052 - val_acc: 0.9431
Epoch 7/10
60000/60000 [==============================] - 3s 52us/step - loss: 0.2492 - acc: 0.9283 - val_loss: 0.1925 - val_a

[I 2019-04-23 12:34:44,316] Finished trial#8 resulted in value: 0.04560000448226931. Current best value is 0.011000003910064726 with parameters: {'n_layer': 3, 'activation': 'relu', 'mid_units': 88.0, 'optimizer': 'adam', 'dropout_rate': 0.0994584167844218}.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 25s 417us/step - loss: 1.3919 - acc: 0.5565 - val_loss: 0.2697 - val_acc: 0.9229
Epoch 2/10
60000/60000 [==============================] - 5s 85us/step - loss: 0.4758 - acc: 0.8512 - val_loss: 0.1339 - val_acc: 0.9587
Epoch 3/10
60000/60000 [==============================] - 5s 86us/step - loss: 0.2807 - acc: 0.9171 - val_loss: 0.0882 - val_acc: 0.9724
Epoch 4/10
60000/60000 [==============================] - 5s 87us/step - loss: 0.1935 - acc: 0.9443 - val_loss: 0.0616 - val_acc: 0.9803
Epoch 5/10
60000/60000 [==============================] - 5s 87us/step - loss: 0.1503 - acc: 0.9562 - val_loss: 0.0536 - val_acc: 0.9826
Epoch 6/10
60000/60000 [==============================] - 5s 87us/step - loss: 0.1229 - acc: 0.9644 - val_loss: 0.0468 - val_acc: 0.9853
Epoch 7/10
60000/60000 [==============================] - 5s 87us/step - loss: 0.1094 - acc: 0.9693 - val_loss: 0.0452 - val_a

[I 2019-04-23 12:35:58,770] Finished trial#9 resulted in value: 0.013199997806549124. Current best value is 0.011000003910064726 with parameters: {'n_layer': 3, 'activation': 'relu', 'mid_units': 88.0, 'optimizer': 'adam', 'dropout_rate': 0.0994584167844218}.


best_params
{'n_layer': 3, 'activation': 'relu', 'mid_units': 88.0, 'optimizer': 'adam', 'dropout_rate': 0.0994584167844218}
-1 x best_value
-0.011000003910064726

 --- sorted --- 

activation : relu
dropout_rate : 0.0994584167844218
mid_units : 88.0
n_layer : 3
optimizer : adam


In [198]:
from keras.models import load_model
model = load_model("my_model.h5")
# model.summary()
test_loss, test_acc =  model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 9s 924us/step


0.9895

In [185]:
'''
モデルの評価
'''

acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, label="Training acc", color= "pink")
plt.plot(epochs, val_acc, label="Validation acc", color= "red")
plt.title("Training and Validation accuracy")
plt.legend()

plt.figure()

plt.plot(epochs, loss, label="Training loss", color= "gray")
plt.plot(epochs, val_loss, label="Validation loss", color= "blue")
plt.title("Training and Validation loss")
plt.legend()

plt.show()

Epoch 1/1
10000/10000 [==============================] - 4s 372us/step - loss: 0.5082 - acc: 0.8727


KeyError: ignored